<h3> CIS4567 Project

<h3> Introduction

With a dataset we obtained from kaggle, https://www.kaggle.com/jboysen/us-perm-visas, we focused on analyzing the decisions and the variables that contribute to the status of a foreign worker to be employed in the United States. We hope to use the columns stated in the dataset to predict decisions for rewarding foreign workers with a visa. With categories such as job positions,country of citizenship, and wages being available as seen in the kaggle website data preview, we hope to use this information to create relationships and predict outcomes. Based on our initial analysis of the dataset on kaggle, we will utilize categorical values and will use chi square tests to analyze the relationships of the categorical values. In our initial analysis, we are attempting to create relationships to the outcome of the decisions. Therefore, we will be using the column "case_status" as our dependent variable with other columns such as "country_of_citzenship" and information about their jobs as independent variables. As we being to analyze the dataset, we will describe the different variables that are necessary in our analysis in further detail. 

<h3> Data Description

Import Libraries

In [1]:
import pyspark
from pyspark.sql import functions as F
import pyspark.mllib.stat as st
import numpy as np
from pyspark.sql.types import *
from sklearn import preprocessing
from pyspark.sql.functions import isnan, when, count, col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574635130169_0002,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Read file

In [2]:
visa_csv = 's3://cis4567fall2019/Project/us_perm_visas.csv'

visa = spark.read.csv(
    visa_csv
    , header=True
    , inferSchema=True
)

for col, typ in visa.dtypes:
    if typ == 'string':
        visa = visa.withColumn(
            col
            , F.ltrim(F.rtrim(visa[col]))
        )
visa.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

374362

View the data types and columns of dataset

In [3]:
visa.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- add_these_pw_job_title_9089: string (nullable = true)
 |-- agent_city: string (nullable = true)
 |-- agent_firm_name: string (nullable = true)
 |-- agent_state: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- case_no: string (nullable = true)
 |-- case_number: string (nullable = true)
 |-- case_received_date: timestamp (nullable = true)
 |-- case_status: string (nullable = true)
 |-- class_of_admission: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- country_of_citzenship: string (nullable = true)
 |-- decision_date: timestamp (nullable = true)
 |-- employer_address_1: string (nullable = true)
 |-- employer_address_2: string (nullable = true)
 |-- employer_city: string (nullable = true)
 |-- employer_country: string (nullable = true)
 |-- employer_decl_info_title: string (nullable = true)
 |-- employer_name: string (nullable = true)
 |-- employer_num_employees: double (nullable = true)
 |-- employer_phone: stri

Keep specific columns for analysis

According to our analysis on the datatypes available and information we find to be relative, we decided to keep columns that we found to be important and relative to our analysis. Although we filtered out the necessary information, we have unfortunately kept only one numerical column, "wage_offer_from_9089". This may hinder our analysis in focusing on different types of analysis but we hope to utilize this single numerical column effectively.

In [4]:
cols_to_keep = visa.dtypes

cols_to_keep = (
                ['case_status', 'class_of_admission', 
                'country_of_citzenship', 'employer_name', 
                'employer_state', 'naics_2007_us_title', 
                'us_economic_sector', 'wage_offer_from_9089', 
                'wage_offer_unit_of_pay_9089'
                ]  
)

cols_to_keep

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['case_status', 'class_of_admission', 'country_of_citzenship', 'employer_name', 'employer_state', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_from_9089', 'wage_offer_unit_of_pay_9089']

Convert 'wage_offer_from_9089' back to double type. In the previous code, it had converted all the columns we wanted to keep as a string so in the next code, we convert the numerical column back to a double.

In [5]:

visa_subset = visa.select(cols_to_keep)

visa_subset_new = visa_subset.withColumn("wage_offer_from_9089",
                                         visa_subset["wage_offer_from_9089"]
                                        .cast(DoubleType()))


visa_subset_new.printSchema()
visa_subset_new.select("wage_offer_from_9089").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- case_status: string (nullable = true)
 |-- class_of_admission: string (nullable = true)
 |-- country_of_citzenship: string (nullable = true)
 |-- employer_name: string (nullable = true)
 |-- employer_state: string (nullable = true)
 |-- naics_2007_us_title: string (nullable = true)
 |-- us_economic_sector: string (nullable = true)
 |-- wage_offer_from_9089: double (nullable = true)
 |-- wage_offer_unit_of_pay_9089: string (nullable = true)

+--------------------+
|wage_offer_from_9089|
+--------------------+
|             75629.0|
|             37024.0|
|             47923.0|
|               10.97|
|            100000.0|
|             37024.0|
|             47084.0|
|             36733.0|
|             44824.0|
|               12.86|
|             60000.0|
|             50000.0|
|                34.3|
|             80000.0|
|             36691.0|
|               15.64|
|               28.05|
|                12.5|
|               10.37|
|                12.0|
+---------------

<h3> Handling Missing Values

Count the number of missing values in all rows in the column 'case_status'(our dependent variable)

In [6]:
visa_subset_new.rdd.map(
        lambda row: (
                row['case_status']
                ,sum([c == None for c in row])
        )
    )\
    .filter(lambda el:el[1] >=1)\
    .count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

355621

Count the number of missing values in each row

In [7]:
(
    spark.createDataFrame(
        visa_subset_new.rdd.map(
           lambda row: (
                 row['case_status']
               , sum([c == None for c in row])
           )
        )
        .filter(lambda el: el[1] >= 1)
        .collect()
        ,['case_status', 'CountMissing']
    )
    .orderBy('CountMissing', ascending=False)
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------+
|case_status|CountMissing|
+-----------+------------+
|  Withdrawn|           8|
|     Denied|           8|
|     Denied|           8|
|     Denied|           8|
|     Denied|           8|
|  Withdrawn|           8|
|  Withdrawn|           8|
|     Denied|           7|
|  Withdrawn|           7|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
|     Denied|           6|
+-----------+------------+
only showing top 20 rows

Count the number of missing values in each column

In [8]:
visa_subset_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in visa_subset_new.columns]).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'str' object is not callable
Traceback (most recent call last):
  File "<stdin>", line 1, in <listcomp>
TypeError: 'str' object is not callable



Count the percentage of missing values in each column

In [9]:
for k, v in sorted(
    visa_subset_new.agg(*[
               (1 - (F.count(c) / F.count('*')))
                    .alias(c + '_missing')
               for c in visa_subset_new.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

naics_2007_us_title_missing 0.9468428953793387
country_of_citzenship_missing 0.9450424989715837
us_economic_sector_missing 0.6552321015487683
wage_offer_from_9089_missing 0.6002692580977771
wage_offer_unit_of_pay_9089_missing 0.30963612759842074
class_of_admission_missing 0.06102382186226163
employer_state_missing 0.00011219087407376982
employer_name_missing 3.205453544963266e-05
case_status_missing 0.0

Although roughly 95% of important columns such as 'country_of_citzenship' and 'naics_2007_us_title' contain missing values, we must drop them in order to keep necessary valuable information regarding our analysis. For our analysis, we hoped to provide a relationship or a correlation analysis of variables such as the country of citizenship so we must ensure data integrity and remove the missing values to try and keep our analysis accurate and consistent.

We drop the missing values and compare the count of the dataset before and after

In [10]:
visa_nonull = visa_subset_new.dropna()
visa_nonull.count(), visa_subset_new.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(18741, 374362)

<h3> Handling Duplicates

Count the number of duplicates in the non-null dataset

In [11]:
visa_nonull.count(), visa_nonull.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(18741, 16503)

Count and show the number of duplicates

In [12]:
(
    visa_nonull
    .groupby(visa_nonull.columns)
    .count()
    .filter('count > 1')
    .show()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---------------------+--------------------+--------------+--------------------+--------------------+--------------------+---------------------------+-----+
|      case_status|class_of_admission|country_of_citzenship|       employer_name|employer_state| naics_2007_us_title|  us_economic_sector|wage_offer_from_9089|wage_offer_unit_of_pay_9089|count|
+-----------------+------------------+---------------------+--------------------+--------------+--------------------+--------------------+--------------------+---------------------------+-----+
|        Withdrawn|               B-2|          SOUTH KOREA|          UTCA, INC.|            CA|Apparel, Piece Go...|Other Economic Se...|            117436.8|                         yr|    2|
|           Denied|               E-1|                ITALY|IMA LIFE NORTH AM...|            NY|All Other Miscell...|        Advanced Mfg|             55848.0|                         yr|    2|
|Certified-Expired|           

We can assume certain variables are fine with being duplicates such as the type of visa or job title but variables such as wages being as specific as they are (first row shows 117436.8), we can assume they are multiple records of the same instance most likely caused by error. Due to this reason, we are dropping duplicates in our dataset.

In [13]:
visa_clean = visa_nonull.dropDuplicates()
visa_clean.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16503

In [14]:
#visa_clean.write.csv('s3://cis4567fall2019/Project/visa_clean.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3> Descriptive Statistics

In [15]:
visa_clean.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('case_status', 'string'), ('class_of_admission', 'string'), ('country_of_citzenship', 'string'), ('employer_name', 'string'), ('employer_state', 'string'), ('naics_2007_us_title', 'string'), ('us_economic_sector', 'string'), ('wage_offer_from_9089', 'double'), ('wage_offer_unit_of_pay_9089', 'string')]

Obtain numerical and categorical columns and define them

In [16]:
cols_num = [
    e[0] for e in visa_clean.dtypes 
    if e[1] == 'double'
]
cols_cat = [
    e[0] for e in visa_clean.dtypes[1:] #we exclude column case_status because it is our dependent variable
    if e[1] == 'string'
]
cols_num, cols_cat

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['wage_offer_from_9089'], ['class_of_admission', 'country_of_citzenship', 'employer_name', 'employer_state', 'naics_2007_us_title', 'us_economic_sector', 'wage_offer_unit_of_pay_9089'])

We extract disinct categories for each column and the count of each catergory

In [17]:
visa_rdd_cat = (
    visa_clean
    .select(cols_cat + ['case_status']) # bring column label back in
    .rdd
    .map(lambda row: [e for e in row])
)

#Create an empty dictionary
visa_results_cat = {}

for i, col in enumerate(cols_cat + ['case_status']):
    visa_results_cat[col] = (
        visa_rdd_cat
        .groupBy(lambda row: row[i]) #group by distinct values in column i
        .map(lambda el: (el[0], len(el[1]))) #map to (category name, count)
        .collect()
    )
    
#print the results
for k in visa_results_cat:
    print(
        k #column name, dictionary key
        , sorted( #sort by frequency of each category, descending
            visa_results_cat[k] #print results
            , key=lambda el: el[1] #use frequency for sorting
            , reverse=True)
        , '\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

class_of_admission [('H-1B', 13655), ('L-1', 514), ('F-1', 480), ('B-2', 293), ('Parolee', 280), ('EWI', 263), ('E-2', 244), ('TN', 146), ('Not in USA', 122), ('H-1B1', 66), ('B-1', 61), ('H-4', 47), ('O-1', 45), ('J-1', 43), ('E-1', 39), ('F-2', 37), ('E-3', 31), ('H-2B', 30), ('L-2', 18), ('A1/A2', 11), ('R-1', 9), ('TPS', 7), ('H-1C', 7), ('C-1', 6), ('VWT', 5), ('P-1', 5), ('H-2A', 5), ('A-3', 4), ('H-1A', 4), ('G-5', 4), ('J-2', 3), ('G-4', 3), ('G-1', 2), ('Parol', 2), ('P-3', 2), ('I', 2), ('R-2', 1), ('D-1', 1), ('H-3', 1), ('O-3', 1), ('P-4', 1), ('Q', 1), ('AOS/H-1B', 1), ('VWB', 1)] 

country_of_citzenship [('INDIA', 8057), ('SOUTH KOREA', 950), ('CANADA', 925), ('CHINA', 922), ('PHILIPPINES', 543), ('MEXICO', 456), ('UNITED KINGDOM', 293), ('TAIWAN', 252), ('PAKISTAN', 237), ('JAPAN', 203), ('TURKEY', 195), ('BRAZIL', 156), ('GERMANY', 154), ('FRANCE', 152), ('NEPAL', 142), ('COLOMBIA', 131), ('VENEZUELA', 131), ('RUSSIA', 106), ('ISRAEL', 102), ('POLAND', 86), ('ITALY', 85

<h3> Statistical Testing

We will now try to analyze the relationships between "case_status" to "country_of_citzenship" and "us_economic_sector". Based on our findings, we hope to see if there is a strong relationship of being 'Certified', 'Certified-Expired', 'Denied', or 'Withdrawn' based on these variables

Group by country of citizenship and show the case status based on highest amount of certified. We must drop the null values in order to proceed with our chi-square analysis

In [18]:
visa_country = (
    visa_clean
    .groupby('country_of_citzenship')
    .pivot('case_status')
    .count()
    .orderBy('Certified', ascending=False)
    .dropna()
)
visa_country.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+---------+-----------------+------+---------+
|country_of_citzenship|Certified|Certified-Expired|Denied|Withdrawn|
+---------------------+---------+-----------------+------+---------+
|                INDIA|     4798|             2012|   685|      562|
|               CANADA|      586|              228|    69|       42|
|                CHINA|      565|              242|    68|       47|
|          SOUTH KOREA|      510|              194|   169|       77|
|          PHILIPPINES|      292|               95|   114|       42|
|               MEXICO|      227|               69|   137|       23|
|       UNITED KINGDOM|      169|               66|    38|       20|
|               TAIWAN|      139|               73|    23|       17|
|             PAKISTAN|      136|               52|    37|       12|
|                JAPAN|      123|               37|    34|        9|
|               TURKEY|      115|               41|    31|        8|
|               BRAZIL|       92| 

Group by economic sector and show the case status based on highest amount of certified. We must drop the null values in order to proceed with our chi-square analysis

In [19]:
visa_sector = (
    visa_clean
    .groupby('us_economic_sector')
    .pivot('case_status')
    .count()
    .orderBy('Certified', ascending=False)
    .dropna()
)
visa_sector.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----------------+------+---------+
|  us_economic_sector|Certified|Certified-Expired|Denied|Withdrawn|
+--------------------+---------+-----------------+------+---------+
|                  IT|     2944|             1423|   569|      448|
|        Advanced Mfg|     1477|              566|   231|      111|
|Other Economic Se...|     1402|              545|   320|      130|
|             Finance|      917|              326|    70|       82|
|Educational Services|      864|              310|   147|       77|
|         Health Care|      718|              248|   145|       77|
|           Aerospace|      361|              146|    48|       30|
|              Retail|      287|              113|    74|       21|
|         Hospitality|      163|               37|   111|       23|
|              Energy|      129|               48|     7|        6|
|        Construction|      114|               49|    87|       22|
|          Automotive|      106|               3

We will now flatmap the data to assist us in further analysis. We have also dropped the null values that are present due to some 'country_of_citzenship' and 'us_economic_sector' not providing actions for those types of 'case_status'. In some examples, a smaller country may not deny or withdraw the visa and in smaller job sectors, we may not see the same case as seen above in "Homeland Security" having no "Withdrawn" visas.

In [20]:
#Flatmap for Country of Citizenship
visa_country_chi = (
    visa_country
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

visa_country_chi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[4798, 2012, 685, 562, 586, 228, 69, 42, 565, 242, 68, 47, 510, 194, 169, 77, 292, 95, 114, 42, 227, 69, 137, 23, 169, 66, 38, 20, 139, 73, 23, 17, 136, 52, 37, 12, 123, 37, 34, 9, 115, 41, 31, 8, 92, 35, 21, 8, 90, 45, 10, 9, 83, 41, 19, 9, 74, 39, 22, 7, 74, 35, 14, 8, 70, 31, 24, 6, 61, 27, 6, 8, 60, 31, 10, 5, 58, 12, 14, 1, 50, 19, 9, 1, 47, 19, 3, 4, 46, 23, 7, 1, 43, 16, 11, 1, 40, 15, 23, 8, 39, 7, 5, 1, 39, 11, 3, 2, 37, 12, 5, 3, 37, 16, 13, 3, 35, 12, 11, 5, 34, 11, 6, 8, 30, 10, 3, 2, 30, 20, 4, 1, 29, 16, 6, 1, 29, 19, 8, 3, 28, 7, 1, 4, 28, 6, 33, 8, 27, 9, 6, 3, 26, 8, 2, 3, 25, 12, 5, 5, 25, 14, 1, 2, 24, 10, 2, 5, 24, 12, 9, 2, 24, 14, 13, 3, 23, 9, 4, 1, 22, 13, 8, 4, 21, 8, 6, 3, 19, 5, 8, 2, 17, 4, 5, 1, 15, 3, 2, 3, 15, 4, 2, 1, 13, 2, 1, 2, 13, 8, 2, 5, 13, 3, 10, 6, 12, 2, 1, 2, 10, 5, 14, 2, 8, 2, 1, 2, 6, 1, 3, 1, 6, 3, 1, 1, 5, 1, 2, 2, 4, 2, 4, 1, 4, 1, 2, 1, 4, 1, 1, 1, 4, 4, 7, 1, 4, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1]

In [21]:
#Flatmap for US Economic Sector
visa_sector_chi = (
    visa_sector
    .rdd
    .map(lambda row: (row[1:]))
    .flatMap(lambda row: row)
    .collect()
)

visa_sector_chi

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2944, 1423, 569, 448, 1477, 566, 231, 111, 1402, 545, 320, 130, 917, 326, 70, 82, 864, 310, 147, 77, 718, 248, 145, 77, 361, 146, 48, 30, 287, 113, 74, 21, 163, 37, 111, 23, 129, 48, 7, 6, 114, 49, 87, 22, 106, 38, 22, 8, 93, 44, 25, 13, 38, 18, 2, 2, 15, 7, 16, 7, 10, 3, 5, 1]

We will now create a DenseMatrix that will be used for our chi-square tests

In [22]:
import pyspark.mllib.linalg as ln
len_row_country = len(visa_country.collect())
len_row_sector = len(visa_sector.collect())
dense_mat_country = ln.DenseMatrix( #parameters: int numRows, int numCols, double[] values, boolean isTransposed
    len_row_country
    , 4
    , visa_country_chi
    , True
)
dense_mat_sector = ln.DenseMatrix( #parameters: int numRows, int numCols, double[] values, boolean isTransposed
    len_row_sector
    , 4
    , visa_sector_chi
    , True
)
dense_mat_country, dense_mat_sector

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(DenseMatrix(67, 4, [4798.0, 2012.0, 685.0, 562.0, 586.0, 228.0, 69.0, 42.0, ..., 3.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0], True), DenseMatrix(16, 4, [2944.0, 1423.0, 569.0, 448.0, 1477.0, 566.0, 231.0, 111.0, ..., 15.0, 7.0, 16.0, 7.0, 10.0, 3.0, 5.0, 1.0], True))

<h3> Chi-Square Tests

In [23]:
chi_sq_country = st.Statistics.chiSqTest(dense_mat_country)

print("Chi-Square test for Country of Citizenship")
print(chi_sq_country.pValue) #check p-value, p < 0.05 means we can reject the null hypothesis
print(chi_sq_country.nullHypothesis) #check the null hypothesis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chi-Square test for Country of Citizenship
0.0
the occurrence of the outcomes is statistically independent.

pValue is 0.0, so <b>we can reject the null hypothesis</b>. This states the
distribution of Country of Citizenship between those that are Certified, Certified-Expired, Denied, and Withdrawn is the same. Therefore, we can conclude that the observed outcome would show evidence that the variables are associated and that the occurrence of the outcomes of 'case_status' is statistically independent and Country of Citizenship should be a strong indicator for those who get Certified, Certified-Expired, Denied, and Withdrawn

In [24]:
chi_sq_sector = st.Statistics.chiSqTest(dense_mat_sector)
print("Chi-Square test for US Economic Sector or Job Role")
print(chi_sq_sector.pValue) #check p-value, p < 0.05 means we can reject the null hypothesis
print(chi_sq_sector.nullHypothesis) #check the null hypothesis

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chi-Square test for US Economic Sector or Job Role
0.0
the occurrence of the outcomes is statistically independent.

<p>pValue is 0.0, so <b>we can reject the null hypothesis</b>. This states the
distribution of US Economic Sector between those that are Certified, Certified-Expired, Denied, and Withdrawn is the same. Therefore, we can conclude that the observed outcome would show evidence that the variables are associated and that the occurrence of the outcomes of 'case_status' is statistically independent and Us Economic Sector should be a strong indicator for those who get Certified, Certified-Expired, Denied, and Withdrawn

<h3> Handling different types of numbers

In order for us to do descriptive statistics for "wage_offer_from_9089", we must separate dollar amount by unit of pay. We filter "wage_offer_unit_of_pay_9089" to separate 'yr', 'hr', 'wk', 'mth', and 'bi'. Based on our findings, we found the 'wage_offer_from_9089' column had values of wages for different types of methods of pay. Because we will not be able to analyze the data accurately, we will separate these different types of wages.

In [25]:
wage_yr = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'yr')
    .select('wage_offer_from_9089')

)

wage_hr = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'hr')
    .select('wage_offer_from_9089')
)

wage_wk = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'wk')
    .select('wage_offer_from_9089')
)

wage_mth = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'mth')
    .select('wage_offer_from_9089')
)

wage_bi = (
    visa_clean
    .filter(visa_clean.wage_offer_unit_of_pay_9089 == 'bi')
    .select('wage_offer_from_9089')
)
wage_yr.show(), wage_hr.show(), wage_wk.show(), wage_mth.show(), wage_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|wage_offer_from_9089|
+--------------------+
|            110000.0|
|            121400.0|
|             60000.0|
|             61260.0|
|             91000.0|
|             18000.0|
|            116438.0|
|             61214.0|
|            105373.0|
|            101000.0|
|            101090.0|
|             70000.0|
|             50000.0|
|             30160.0|
|             68349.0|
|             85551.0|
|             80850.0|
|            112000.0|
|             64792.0|
|            350000.0|
+--------------------+
only showing top 20 rows

+--------------------+
|wage_offer_from_9089|
+--------------------+
|               10.16|
|                25.0|
|                80.0|
|               12.35|
|               13.56|
|                80.0|
|                34.0|
|               12.21|
|                80.0|
|                65.0|
|                 8.0|
|               19.31|
|               12.18|
|               15.81|
|                81.0|
|       

In [27]:
'''features = ['wage_offer_from_9089']
quantiles = [0.25, 0.75]

cut_off_points = []

#getting Q1 and Q3
#The third parameter, set to 0.05, is relative target precision to achieve (>= 0).
#If set to zero, the exact quantiles are computed, which could be very expensive. 
for feature in features:
    quants_yr = wage_yr.approxQuantile(feature, quantiles, 0.05)
    quants_hr = wage_hr.approxQuantile(feature, quantiles, 0.05)
    quants_wk = wage_wk.approxQuantile(feature, quantiles, 0.05)
    quants_mth = wage_mth.approxQuantile(feature, quantiles, 0.05)
    quants_bi = wage_bi.approxQuantile(feature, quantiles, 0.05)
    print(feature, quants_yr)
    print(feature, quants_hr)
    print(feature, quants_wk)
    print(feature, quants_mth)
    print(feature, quants_bi)
    IQR_yr = quants_yr[1] - quants_yr[0]
    cut_off_points.append((feature, [
        quants_yr[0] - 1.5 * IQR_yr,
        quants_yr[1] + 1.5 * IQR_yr,
    ]))
    IQR_hr = quants_hr[1] - quants_hr[0]
    cut_off_points.append((feature, [
        quants_hr[0] - 1.5 * IQR_hr,
        quants_hr[1] + 1.5 * IQR_hr,
    ]))
    IQR_wk = quants_wk[1] - quants_wk[0]
    cut_off_points.append((feature, [
        quants_wk[0] - 1.5 * IQR_wk,
        quants_wk[1] + 1.5 * IQR_wk,
    ]))
    IQR_mth = quants_mth[1] - quants_mth[0]
    cut_off_points.append((feature, [
        quants_mth[0] - 1.5 * IQR_mth,
        quants_mth[1] + 1.5 * IQR_mth,
    ]))
    IQR_bi = quants_bi[1] - quants_bi[0]
    cut_off_points.append((feature, [
        quants_bi[0] - 1.5 * IQR_bi,
        quants_bi[1] + 1.5 * IQR_bi,
    ]))

cut_off_points = dict(cut_off_points)

outliers_yr = wage_yr.select(*['wage_offer_from_9089'] + [
       (
           (wage_yr[F] < cut_off_points[F][0]) |
           (wage_yr[F] > cut_off_points[F][1])
       ).alias(F + '_o') for F in features
  ])
outliers_hr = wage_hr.select(*['wage_offer_from_9089'] + [
       (
           (wage_hr[F] < cut_off_points[F][0]) |
           (wage_hr[F] > cut_off_points[F][1])
       ).alias(F + '_o') for F in features
  ])
outliers_wk = wage_wk.select(*['wage_offer_from_9089'] + [
       (
           (wage_wk[F] < cut_off_points[F][0]) |
           (wage_wk[F] > cut_off_points[F][1])
       ).alias(F + '_o') for F in features
  ])
outliers_mth = wage_mth.select(*['wage_offer_from_9089'] + [
       (
           (wage_mth[F] < cut_off_points[F][0]) |
           (wage_mth[F] > cut_off_points[F][1])
       ).alias(F + '_o') for F in features
  ])
outliers_bi = wage_bi.select(*['wage_offer_from_9089'] + [
       (
           (wage_bi[F] < cut_off_points[F][0]) |
           (wage_bi[F] > cut_off_points[F][1])
       ).alias(F + '_o') for F in features
  ])
outliers_yr.show(),outliers_hr.show(),outliers_wk.show(),outliers_mth.show(),outliers_bi.show()'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"features = ['wage_offer_from_9089']\nquantiles = [0.25, 0.75]\n\ncut_off_points = []\n\n#getting Q1 and Q3\n#The third parameter, set to 0.05, is relative target precision to achieve (>= 0).\n#If set to zero, the exact quantiles are computed, which could be very expensive. \nfor feature in features:\n    quants_yr = wage_yr.approxQuantile(feature, quantiles, 0.05)\n    quants_hr = wage_hr.approxQuantile(feature, quantiles, 0.05)\n    quants_wk = wage_wk.approxQuantile(feature, quantiles, 0.05)\n    quants_mth = wage_mth.approxQuantile(feature, quantiles, 0.05)\n    quants_bi = wage_bi.approxQuantile(feature, quantiles, 0.05)\n    print(feature, quants_yr)\n    print(feature, quants_hr)\n    print(feature, quants_wk)\n    print(feature, quants_mth)\n    print(feature, quants_bi)\n    IQR_yr = quants_yr[1] - quants_yr[0]\n    cut_off_points.append((feature, [\n        quants_yr[0] - 1.5 * IQR_yr,\n        quants_yr[1] + 1.5 * IQR_yr,\n    ]))\n    IQR_hr = quants_hr[1] - quants_hr[0]\n 

<h3> Handling outliers

In order for us to create accurate histograms for our only numerical column, we will have to create a function to identify the outliers then proceed to remove those outliers to give an accurate representation of our dataset via histogram. Below, we calculate the cut off points of the outliers in a for-loop of the different wage types we have. We will then create a variable for each wage type to a specific outlier variable to match them. This will help us later when we create individual histograms for each wage type.

In [28]:
features = ['wage_offer_from_9089']
quantiles = [0.25, 0.75]

cut_off_points = []
wage_types = [wage_yr, wage_hr, wage_wk, wage_mth, wage_bi]
#getting Q1 and Q3
#The third parameter, set to 0.05, is relative target precision to achieve (>= 0).
#If set to zero, the exact quantiles are computed, which could be very expensive.
for wages in wage_types:
    for feature in features:
        quants = wages.approxQuantile(feature, quantiles, 0.05)
        print(feature, quants)
        IQR = quants[1] - quants[0]
        cut_off_points.append((feature, [
            quants[0] - 1.5 * IQR,
            quants[1] + 1.5 * IQR,
        ]))

    cut_off_points2 = dict(cut_off_points)


    outliers_yr = wage_yr.select(*['wage_offer_from_9089'] + [
           (
               (wage_yr[F] < cut_off_points2[F][0]) |
               (wage_yr[F] > cut_off_points2[F][1])
           ).alias(F + '_o') for F in features
      ])
    outliers_hr = wage_hr.select(*['wage_offer_from_9089'] + [
           (
               (wage_hr[F] < cut_off_points2[F][0]) |
               (wage_hr[F] > cut_off_points2[F][1])
           ).alias(F + '_o') for F in features
      ])
    outliers_wk = wage_wk.select(*['wage_offer_from_9089'] + [
           (
               (wage_wk[F] < cut_off_points2[F][0]) |
               (wage_wk[F] > cut_off_points2[F][1])
           ).alias(F + '_o') for F in features
      ])
    outliers_mth = wage_mth.select(*['wage_offer_from_9089'] + [
           (
               (wage_mth[F] < cut_off_points2[F][0]) |
               (wage_mth[F] > cut_off_points2[F][1])
           ).alias(F + '_o') for F in features
      ])
    outliers_bi = wage_bi.select(*['wage_offer_from_9089'] + [
           (
               (wage_bi[F] < cut_off_points2[F][0]) |
               (wage_bi[F] > cut_off_points2[F][1])
           ).alias(F + '_o') for F in features
      ])

outliers_yr.show(),outliers_hr.show(),outliers_wk.show(),outliers_mth.show(),outliers_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_offer_from_9089 [80004.0, 120000.0]
wage_offer_from_9089 [21.88, 49.0]
wage_offer_from_9089 [1400.0, 2540.0]
wage_offer_from_9089 [4576.0, 7200.0]
wage_offer_from_9089 [1200.0, 2000.0]
+--------------------+----------------------+
|wage_offer_from_9089|wage_offer_from_9089_o|
+--------------------+----------------------+
|            110000.0|                  true|
|            121400.0|                  true|
|             60000.0|                  true|
|             61260.0|                  true|
|             91000.0|                  true|
|             18000.0|                  true|
|            116438.0|                  true|
|             61214.0|                  true|
|            105373.0|                  true|
|            101000.0|                  true|
|            101090.0|                  true|
|             70000.0|                  true|
|             50000.0|                  true|
|             30160.0|                  true|
|             68349.0|      

We now filter and select the rows containing the outliers

In [29]:
with_outliers_flag_yr = wage_yr.join(outliers_yr, on = 'wage_offer_from_9089')
(
    with_outliers_flag_yr
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_hr = wage_hr.join(outliers_hr, on = 'wage_offer_from_9089')
(
    with_outliers_flag_hr
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_wk = wage_wk.join(outliers_wk, on = 'wage_offer_from_9089')
(
    with_outliers_flag_wk
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_mth = wage_mth.join(outliers_mth, on = 'wage_offer_from_9089')
(
    with_outliers_flag_mth
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)
with_outliers_flag_bi = wage_bi.join(outliers_bi, on = 'wage_offer_from_9089')
(
    with_outliers_flag_bi
    .filter('wage_offer_from_9089_o')
    .select('wage_offer_from_9089')
    .show()
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|wage_offer_from_9089|
+--------------------+
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
|            110000.0|
+--------------------+
only showing top 20 rows

+--------------------+
|wage_offer_from_9089|
+--------------------+
|             68940.0|
|            158828.8|
|             72675.0|
|             86070.0|
|             42078.0|
+--------------------+

+--------------------+
|wage_offer_from_9089|
+--------------------+
|            110240.0|
|            103030.0|
|            224000.0|
|             86000.0|
+--------------------+

+--------------------+
|wage_

We filter the outliers of each specific wage type in the code below. This is necessary to create variables for instances where we do not have outliers. With outliers, we will not have an accurate representation in our histogram. With outliers now filtered out, we can hope to create histograms to represent the wages by wage types.

In [30]:
no_outliers_yr = (
    with_outliers_flag_yr
    .filter('!wage_offer_from_9089_o')
    .select(wage_yr.columns)
)
no_outliers_hr = (
    with_outliers_flag_hr
    .filter('!wage_offer_from_9089_o')
    .select(wage_hr.columns)
)
no_outliers_wk = (
    with_outliers_flag_wk
    .filter('!wage_offer_from_9089_o')
    .select(wage_wk.columns)
)
no_outliers_mth = (
    with_outliers_flag_mth
    .filter('!wage_offer_from_9089_o')
    .select(wage_mth.columns)
)
no_outliers_bi = (
    with_outliers_flag_bi
    .filter('!wage_offer_from_9089_o')
    .select(wage_bi.columns)
)
no_outliers_yr.show(), no_outliers_hr.show(), no_outliers_wk.show(), no_outliers_mth.show(), no_outliers_bi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|wage_offer_from_9089|
+--------------------+
+--------------------+

+--------------------+
|wage_offer_from_9089|
+--------------------+
|               10.16|
|                25.0|
|                25.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
|                80.0|
+--------------------+
only showing top 20 rows

+--------------------+
|wage_offer_from_9089|
+--------------------+
|              1114.7|
|               940.0|
|             1730.77|
|              2189.0|
|              2540.0|
|             1574.29|
|               908.8|
|               900.0|
|              1400.0|
|              2520.0|
|             1587.55|
|      

In [45]:
descriptive_stats = no_outliers_yr.describe(features)
descriptive_stats.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|summary|wage_offer_from_9089|
+-------+--------------------+
|  count|                   0|
|   mean|                null|
| stddev|                null|
|    min|                null|
|    max|                null|
+-------+--------------------+

Based on our findings, we see that all values in the wage_yr column were outliers. This is a problem that we must identify and address. We must do further analysis as to why this whole column was considered an outlier. 

We will now create bins and the count of each bin in order for us to create our histogram

In [32]:
histogram_wage_hr = (
    no_outliers_hr
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_wk = (
    no_outliers_wk
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_mth = (
    no_outliers_mth
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)
histogram_wage_bi = (
    no_outliers_bi
    .select('wage_offer_from_9089')
    .rdd
    .flatMap(lambda x: x)
    .histogram(5)
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
for a in histogram_wage_hr:
    print('wage_hr',a)    
for b in histogram_wage_wk:
    print('wage_wk',b)    
for c in histogram_wage_mth:
    print('wage_mth',c)    
for d in histogram_wage_bi:
    print('wage_bi',d)    
histogram_wage_hr, histogram_wage_wk, histogram_wage_mth, histogram_wage_bi


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_hr [7.62, 36.096, 64.572, 93.048, 121.524, 150.0]
wage_hr [566, 292, 980, 18, 1]
wage_wk [900.0, 1228.0, 1556.0, 1884.0, 2212.0, 2540.0]
wage_wk [4, 2, 5, 2, 2]
wage_mth [2600.0, 2635.2, 2670.4, 2705.6, 2740.8, 2776.0]
wage_mth [1, 0, 0, 0, 1]
wage_bi [1033.6, 1253.78, 1473.96, 1694.1399999999999, 1914.32, 2134.5]
wage_bi [2, 1, 0, 0, 2]
(([7.62, 36.096, 64.572, 93.048, 121.524, 150.0], [566, 292, 980, 18, 1]), ([900.0, 1228.0, 1556.0, 1884.0, 2212.0, 2540.0], [4, 2, 5, 2, 2]), ([2600.0, 2635.2, 2670.4, 2705.6, 2740.8, 2776.0], [1, 0, 0, 0, 1]), ([1033.6, 1253.78, 1473.96, 1694.1399999999999, 1914.32, 2134.5], [2, 1, 0, 0, 2]))

In [34]:
#display bin ranges
for a in range(len(histogram_wage_hr[0])-1):
    print('wage_hr','[' + str(round(histogram_wage_hr[0][a],2))
        + ',' + str(round(histogram_wage_hr[0][a+1],2))
          + ')'
         )
for b in range(len(histogram_wage_wk[0])-1):
    print('wage_wk','[' + str(round(histogram_wage_wk[0][b],2))
        + ',' + str(round(histogram_wage_wk[0][b+1],2))
          + ')'
         )
for c in range(len(histogram_wage_mth[0])-1):
    print('wage_mth','[' + str(round(histogram_wage_mth[0][c],2))
        + ',' + str(round(histogram_wage_mth[0][c+1],2))
          + ')'
         )
for d in range(len(histogram_wage_bi[0])-1):
    print('wage_bi','[' + str(round(histogram_wage_bi[0][d],2))
        + ',' + str(round(histogram_wage_bi[0][d+1],2))
          + ')'
         )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

wage_hr [7.62,36.1)
wage_hr [36.1,64.57)
wage_hr [64.57,93.05)
wage_hr [93.05,121.52)
wage_hr [121.52,150.0)
wage_wk [900.0,1228.0)
wage_wk [1228.0,1556.0)
wage_wk [1556.0,1884.0)
wage_wk [1884.0,2212.0)
wage_wk [2212.0,2540.0)
wage_mth [2600.0,2635.2)
wage_mth [2635.2,2670.4)
wage_mth [2670.4,2705.6)
wage_mth [2705.6,2740.8)
wage_mth [2740.8,2776.0)
wage_bi [1033.6,1253.78)
wage_bi [1253.78,1473.96)
wage_bi [1473.96,1694.14)
wage_bi [1694.14,1914.32)
wage_bi [1914.32,2134.5)

In [35]:
sorted(zip(*histogram_wage_hr))
sorted(zip(*histogram_wage_wk))
sorted(zip(*histogram_wage_mth))
sorted(zip(*histogram_wage_bi))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1033.6, 2), (1253.78, 1), (1473.96, 0), (1694.1399999999999, 0), (1914.32, 2)]

In [36]:
%%spark -o hist_wage_hr
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_hr=  spark.createDataFrame(
    list(zip(*histogram_wage_hr)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
%%spark -o hist_wage_wk
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_wk=  spark.createDataFrame(
    list(zip(*histogram_wage_wk)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
%%spark -o hist_wage_mth
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_mth=  spark.createDataFrame(
    list(zip(*histogram_wage_mth)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
%%spark -o hist_wage_bi
#The above Spark magic exports hist_wage to local node running the notebook
#So that we can use the local object to draw a histogram
import pandas as pd
hist_wage_bi=  spark.createDataFrame(
    list(zip(*histogram_wage_bi)), 
    ['bins', 'counts'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Histogram for wage_hr

In [40]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_hr['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Hourly Wage'
)

p.vbar(x=labels, top=hist_wage_hr['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p>Based on the histogram, we can see that the majority of people fall within \$64.57 for an hourly wage with a count of about 1000. We can also see that almost 600 people fall within the \$7.62/hr range with a very small amount of people above the \$93.05/hr bin range. This histogram can be identified as skewing left

<h3>Histogram for wage_wk

In [41]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_wk['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Weekly Wage'
)

p.vbar(x=labels, top=hist_wage_wk['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p> Based on the histogram, we are able to see 5 people who get their wages weekly fall within the bin range of \$1556.00/wk. We are not able to create a very accurate representation on wages because not many people fall within employees who receive their wages weekly.

<h3>Histogram for wage_mth

In [42]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_mth['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Monthly Wage'
)

p.vbar(x=labels, top=hist_wage_mth['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p>As stated previously, due to our low sample size, we are not able to create an analysis to relate to our dataset. We can see that the ranges between the two people who receive wages as monthly, are close to eachother. One person falls within the bin range of \$2600.00 while the other falls within the bin range of \$2740.80.

<h3>Histogram for wage_bi

In [43]:
%%local
#local magic indicates that this cell will run locally (rather than on cluster)
#use Bokeh library ti draw the histogram
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
output_notebook()

labels = [str(round(e, 2)) for e in hist_wage_bi['bins']]

p = figure(
    x_range=labels, 
    plot_height=350, 
    title='Histogram of Bi-Weekly Wage'
)

p.vbar(x=labels, top=hist_wage_bi['counts'], width=0.9)

show(p)

Loading BokehJS ...

<p> In the histogram above, we are looking at identifiers for wage_bi. "Bi" as stated in the dataset wasn't accurately described. With these numbers, we can assume they are bi-weekly meaning every 2 weeks worth of pay. Two people fall within bin ranges of \$1033.60 and \$1914.32 as a bi-weekly wage respectively while only one falls within the bin range of \$1253.78 bi-weekly

In [44]:
#code for dummies for future use
'''categ = visa_nonull.select('case_status').distinct().rdd.flatMap(lambda x:x).collect()
exprs = [F.when(F.col('case_status') == cat,1).otherwise(0)\
            .alias(str(cat)) for cat in categ]
visa_nonull_dummy = visa_nonull.select(exprs+visa_nonull.columns)
visa_nonull_dummy.show()
'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"categ = visa_nonull.select('case_status').distinct().rdd.flatMap(lambda x:x).collect()\nexprs = [F.when(F.col('case_status') == cat,1).otherwise(0)            .alias(str(cat)) for cat in categ]\nvisa_nonull_dummy = visa_nonull.select(exprs+visa_nonull.columns)\nvisa_nonull_dummy.show()\n"